# Task 0.2.2

### Prepare a CNN of your choice and train it on the MNIST data. Report the accuracy

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# Save the model
model.save("mnist_model.h5")

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)


2024-04-14 11:34:42.842085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/qasim/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 38ms/step - accuracy: 0.8658 - loss: 0.4377 - val_accuracy: 0.9861 - val_loss: 0.0434
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 37s 40ms/step - accuracy: 0.9822 - loss: 0.0574 - val_accuracy: 0.9877 - val_loss: 0.0371
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 37s 39ms/step - accuracy: 0.9889 - loss: 0.0366 - val_accuracy: 0.9831 - val_loss: 0.0502
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 39s 37ms/step - accuracy: 0.9904 - loss: 0.0291 - val_accuracy: 0.9907 - val_loss: 0.0326
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 39s 42ms/step - accuracy: 0.9927 - loss: 0.0230 - val_accuracy: 0.9896 - val_loss: 0.0310


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9877 - loss: 0.0349
Test accuracy: 0.9896000027656555


### Use the above model as a pre-trained CNN for the SVHN dataset. Report the accuracy

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import cv2
from scipy.io import loadmat
import numpy as np

# Load SVHN dataset
train_data = loadmat('train_32x32.mat')
test_data = loadmat('test_32x32.mat')

# Extract images and labels from SVHN dataset
train_images = np.moveaxis(train_data['X'], -1, 0)
train_labels = train_data['y']
test_images = np.moveaxis(test_data['X'], -1, 0)
test_labels = test_data['y']

# Preprocess SVHN dataset
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Resize SVHN images to 28x28
train_images_resized = [cv2.resize(img, (28, 28)) for img in train_images]
test_images_resized = [cv2.resize(img, (28, 28)) for img in test_images]

# Convert SVHN images to grayscale
train_images_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in train_images_resized]
test_images_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in test_images_resized]

# Expand dimensions to match input shape of the pre-trained MNIST model
train_images_gray = np.expand_dims(train_images_gray, axis=-1)
test_images_gray = np.expand_dims(test_images_gray, axis=-1)

# Convert SVHN labels to one-hot encoding
train_labels = to_categorical(train_labels - 1, num_classes=10)
test_labels = to_categorical(test_labels - 1, num_classes=10)

# Load the pre-trained MNIST model
mnist_model = load_model("mnist_model.h5")

# Modify the output layer for SVHN dataset (10 x classes)
mnist_model.pop()
mnist_model.add(layers.Dense(10, activation='softmax'))

# Compile the model
mnist_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# Evaluate the model on SVHN test data
test_loss, test_acc = mnist_model.evaluate(test_images_gray, test_labels)

print('Test accuracy on SVHN dataset:', test_acc)


814/814 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1138 - loss: 2.6142
Test accuracy on SVHN dataset: 0.11524277925491333


### In the third step you are performing transfer learning from MNIST to SVHN (optional).

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import cv2
from scipy.io import loadmat
import numpy as np

# Load SVHN dataset
train_data = loadmat('train_32x32.mat')
test_data = loadmat('test_32x32.mat')

# Extract images and labels from SVHN dataset
train_images = np.moveaxis(train_data['X'], -1, 0)
train_labels = train_data['y']
test_images = np.moveaxis(test_data['X'], -1, 0)
test_labels = test_data['y']

# Preprocess SVHN dataset
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Resize SVHN images to 28x28
train_images_resized = [cv2.resize(img, (28, 28)) for img in train_images]
test_images_resized = [cv2.resize(img, (28, 28)) for img in test_images]

# Convert SVHN images to grayscale
train_images_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in train_images_resized]
test_images_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in test_images_resized]

# Expand dimensions to match input shape of the pre-trained MNIST model
train_images_gray = np.expand_dims(train_images_gray, axis=-1)
test_images_gray = np.expand_dims(test_images_gray, axis=-1)

# Convert SVHN labels to one-hot encoding
train_labels = to_categorical(train_labels - 1, num_classes=10)
test_labels = to_categorical(test_labels - 1, num_classes=10)

# Load the pre-trained MNIST model
mnist_model = load_model("mnist_model.h5")

# Fine-tune the entire model
for layer in mnist_model.layers:
    layer.trainable = True

# Compile the model
mnist_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# Train the model on SVHN data
history = mnist_model.fit(train_images_gray, train_labels, epochs=5, batch_size=64, validation_data=(test_images_gray, test_labels))

# Evaluate the model on SVHN test data
test_loss, test_acc = mnist_model.evaluate(test_images_gray, test_labels)

print('Test accuracy on SVHN dataset:', test_acc)


Epoch 1/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 48s 40ms/step - accuracy: 0.6513 - loss: 1.0871 - val_accuracy: 0.8534 - val_loss: 0.5085
Epoch 2/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 46s 40ms/step - accuracy: 0.8727 - loss: 0.4345 - val_accuracy: 0.8719 - val_loss: 0.4455
Epoch 3/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 40s 35ms/step - accuracy: 0.8959 - loss: 0.3563 - val_accuracy: 0.8840 - val_loss: 0.4058
Epoch 4/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 43s 38ms/step - accuracy: 0.9070 - loss: 0.3132 - val_accuracy: 0.8968 - val_loss: 0.3579
Epoch 5/5
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 46s 40ms/step - accuracy: 0.9173 - loss: 0.2810 - val_accuracy: 0.9008 - val_loss: 0.3465
814/814 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8976 - loss: 0.3565
Test accuracy on SVHN dataset: 0.9007759690284729
